# Network Setting 1

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
import tensorflow as tf

train = pd.read_csv('train_new_3.csv')
train = train.fillna(0)

test = pd.read_csv('test_new_3.csv')
test = test.fillna(0)

test_id = test['Id']
test = test.drop(columns = ['Id', 'text', 'reply_to_screen_name', 'hashtags', 'clean_tweet'], axis = 1)


## Defining input and target
X = train.drop(columns = ['text', 'reply_to_screen_name', 'hashtags', 'clean_tweet', 'country'], axis = 1)
Y = train['country']
Y = np.where(Y == 'us', 0, 
             np.where(Y == 'uk', 1, 
                      np.where(Y == 'canada', 2, 
                               np.where(Y == 'australia', 3,
                                        np.where(Y == 'ireland', 4, 5)))))

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

## Scaling the data 
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
test = scaler.fit_transform(test)

## Defining model 
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(10, input_dim = 213, activation = 'relu'),
#         tf.keras.layers.Dense(10, activation = 'relu'),
        tf.keras.layers.Dense(6, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Fitting model 
model.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 6), epochs = 20, batch_size = 32, validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 6)))

## Predicting on test
nn_pred = model.predict(test)
nn_pred = np.argmax(nn_pred, axis = 1)

## Defining data to be exported
data_out = pd.DataFrame({'Id': test_id, 'Category': nn_pred})
data_out['Category'] = np.where(data_out['Category'] == 0, 'us',
                                np.where(data_out['Category'] == 1, 'uk',
                                         np.where(data_out['Category'] == 2, 'canada',
                                                  np.where(data_out['Category'] == 3, 'australia',
                                                           np.where(data_out['Category'] == 4, 'ireland', 'new_zealand')))))
data_out.to_csv('nn_submission_md10.csv', index = False)

2022-09-13 11:19:56.409540: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-13 11:19:56.409616: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-16-72-173.ec2.internal): /proc/driver/nvidia/version does not exist
2022-09-13 11:19:56.411920: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
6000/6000 [==============================] - 12s 2ms/step - loss: 1.3600 - accuracy: 0.3639 - val_loss: 1.3070 - val_accuracy: 0.3886
Epoch 2/20
6000/6000 [==============================] - 8s 1ms/step - loss: 1.2973 - accuracy: 0.3925 - val_loss: 1.3054 - val_accuracy: 0.3850
Epoch 3/20
6000/6000 [==============================] - 7s 1ms/step - loss: 1.2929 - accuracy: 0.3939 - val_loss: 1.2968 - val_accuracy: 0.3925
Epoch 4/20
6000/6000 [==============================] - 8s 1ms/step - loss: 1.2910 - accuracy: 0.3937 - val_loss: 1.2921 - val_accuracy: 0.3930
Epoch 5/20
6000/6000 [==============================] - 8s 1ms/step - loss: 1.2893 - accuracy: 0.3961 - val_loss: 1.2916 - val_accuracy: 0.3908
Epoch 6/20
6000/6000 [==============================] - 9s 2ms/step - loss: 1.2881 - accuracy: 0.3965 - val_loss: 1.2960 - val_accuracy: 0.3934
Epoch 7/20
6000/6000 [==============================] - 9s 1ms/step - loss: 1.2871 - accuracy: 0.3959 - val_loss: 1.2948 - val_accuracy

In [5]:
nn_pred = model.predict(X_test)
nn_pred = np.argmax(nn_pred, axis = 1)

## Defining data to be exported
data_out = pd.DataFrame({'Actual': Y_test, 'Pred': nn_pred})
data_out['Actual'] = np.where(data_out['Actual'] == 0, 'us',
                              np.where(data_out['Actual'] == 1, 'uk',
                                       np.where(data_out['Actual'] == 2, 'canada',
                                                np.where(data_out['Actual'] == 3, 'australia',
                                                         np.where(data_out['Actual'] == 4, 'ireland', 'new_zealand')))))

data_out['Pred'] = np.where(data_out['Pred'] == 0, 'us',
                            np.where(data_out['Pred'] == 1, 'uk',
                                     np.where(data_out['Pred'] == 2, 'canada',
                                              np.where(data_out['Pred'] == 3, 'australia',
                                                       np.where(data_out['Pred'] == 4, 'ireland', 'new_zealand')))))
data_out.head()

,Actual,Pred
0,us,australia
1,canada,uk
2,uk,uk
3,canada,canada
4,australia,new_zealand


In [5]:
pd.crosstab(data_out['Actual'], data_out['Pred'])

Pred,canada,ireland,new_zealand,uk,us
Actual,,,,,
australia,0,3261,17467,5342,5950
canada,1,0,0,15029,16851
ireland,0,5066,23214,2136,1598
new_zealand,0,4208,24419,1658,1730
uk,0,0,0,15536,16471
us,0,0,0,14245,17818


In [21]:
pd.crosstab(data_out['Actual'], data_out['Pred'])

Pred,australia,canada,ireland,new_zealand,uk,us
Actual,,,,,,
australia,684,828,2272,2317,888,1127
canada,2,2534,0,0,2879,2474
ireland,417,217,4121,2633,395,314
new_zealand,508,211,3072,3589,269,317
uk,2,2090,0,0,3222,2707
us,0,1921,0,0,2112,3878


In [24]:
pd.crosstab(data_out['Actual'], data_out['Pred'])

Pred,australia,canada,ireland,new_zealand,uk,us
Actual,,,,,,
australia,791,1166,2068,2455,393,1144
canada,12,3836,2,0,1405,2750
ireland,560,362,3790,2817,264,328
new_zealand,585,336,2648,3788,141,372
uk,12,3485,1,0,1626,2810
us,13,2856,0,0,1001,4183


In [6]:
pd.crosstab(data_out['Actual'], data_out['Pred'])

Pred,australia,canada,ireland,new_zealand,uk,us
Actual,,,,,,
australia,1123,364,1495,2807,1240,925
canada,550,1438,17,0,4004,2075
ireland,644,94,3455,3154,513,251
new_zealand,820,99,1961,4560,384,276
uk,452,970,22,0,4408,2055
us,435,866,13,0,2846,3684


In [6]:
data_out['Actual'].value_counts()

us             32063
australia      32020
new_zealand    32015
ireland        32014
uk             32007
canada         31881
Name: Actual, dtype: int64

In [7]:
from sklearn.metrics import classification_report

target_names = ['us', 'uk', 'canada', 'australia', 'ireland', 'new_zealand']
print(classification_report(data_out['Actual'], data_out['Pred'], target_names = target_names))

              precision    recall  f1-score   support

          us       0.28      0.14      0.19      7954
          uk       0.38      0.18      0.24      8084
      canada       0.50      0.43      0.46      8111
   australia       0.43      0.56      0.49      8100
     ireland       0.33      0.56      0.41      7907
 new_zealand       0.40      0.47      0.43      7844

    accuracy                           0.39     48000
   macro avg       0.39      0.39      0.37     48000
weighted avg       0.39      0.39      0.37     48000

